In [1]:
import cv2
import numpy as np
import pygame
import time
import mediapipe as mp
import math
from scipy.spatial import distance as dist

# Initialize mediapipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)

# Initialize Pygame for alert sounds
pygame.mixer.init()
primary_alert = pygame.mixer.Sound("primary_alert.wav")  # Primary alert sound
secondary_alert = pygame.mixer.Sound("secondary_alert.wav")  # Secondary alert sound
alcohol_alert = pygame.mixer.Sound("alcohol_alert.wav")  # Alcohol alert sound

# Eye landmarks for EAR calculations
LEFT_EYE = [362, 385, 387, 263, 373, 380]
RIGHT_EYE = [33, 160, 158, 133, 153, 144]

# Thresholds and counters for drowsiness
EAR_THRESHOLD = 0.2
CLOSED_EYE_DURATION_THRESHOLD = 0.75  # Seconds
HEAD_TILT_THRESHOLD = 15.0  # degrees
HEAD_TILT_DURATION = 5.0  # seconds

# Alcohol detection thresholds
MAR_THRESHOLD = 0.5
HEAD_YAW_THRESHOLD = 15
HEAD_PITCH_THRESHOLD = 30
ALCOHOL_ALERT_THRESHOLD = 2

# Initialize variables
blink_count = 0
start_drowsy_time = None
is_drowsy = False
alert_count = 0


# Function to calculate Eye Aspect Ratio (EAR)
def calculate_ear(eye_landmarks):
    vertical_1 = dist.euclidean(eye_landmarks[1], eye_landmarks[5])
    vertical_2 = dist.euclidean(eye_landmarks[2], eye_landmarks[4])
    horizontal = dist.euclidean(eye_landmarks[0], eye_landmarks[3])
    return (vertical_1 + vertical_2) / (2.0 * horizontal)


# Function to calculate Mouth Aspect Ratio (MAR)
def calculate_mar(landmarks):
    upper_lip = np.array([landmarks.landmark[13].x, landmarks.landmark[13].y])
    lower_lip = np.array([landmarks.landmark[14].x, landmarks.landmark[14].y])
    left_corner = np.array([landmarks.landmark[61].x, landmarks.landmark[61].y])
    right_corner = np.array([landmarks.landmark[291].x, landmarks.landmark[291].y])
    vertical = np.linalg.norm(upper_lip - lower_lip)
    horizontal = np.linalg.norm(left_corner - right_corner)
    return vertical / horizontal


# Function to calculate head pose (yaw and pitch)
def get_head_pose(landmarks):
    nose = np.array([landmarks.landmark[1].x, landmarks.landmark[1].y])
    chin = np.array([landmarks.landmark[152].x, landmarks.landmark[152].y])
    left_eye = np.array([landmarks.landmark[33].x, landmarks.landmark[33].y])
    right_eye = np.array([landmarks.landmark[362].x, landmarks.landmark[362].y])
    yaw = math.degrees(math.atan2(right_eye[1] - left_eye[1], right_eye[0] - left_eye[0]))
    pitch = math.degrees(math.atan2(chin[1] - nose[1], chin[0] - nose[0]))
    return yaw, pitch


# Function to check for alcohol detection
def alcohol_detection_logic(mar, head_yaw, head_pitch):
    criteria_met = 0
    if mar > MAR_THRESHOLD:
        criteria_met += 1
    if abs(head_yaw) > HEAD_YAW_THRESHOLD or abs(head_pitch) > HEAD_PITCH_THRESHOLD:
        criteria_met += 1
    return criteria_met >= ALCOHOL_ALERT_THRESHOLD


# Start capturing video
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Failed to capture image")
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)
    frame_height, frame_width = frame.shape[:2]

    # Initialize display variables
    drowsiness_detected = False
    alcohol_detected = False
    display_text = []

    if results.multi_face_landmarks:
        for landmarks in results.multi_face_landmarks:
            # Drowsiness Detection
            left_eye = [(landmarks.landmark[i].x * frame_width, landmarks.landmark[i].y * frame_height) for i in LEFT_EYE]
            right_eye = [(landmarks.landmark[i].x * frame_width, landmarks.landmark[i].y * frame_height) for i in RIGHT_EYE]
            avg_ear = (calculate_ear(left_eye) + calculate_ear(right_eye)) / 2.0
            if avg_ear < EAR_THRESHOLD:
                if start_drowsy_time is None:
                    start_drowsy_time = time.time()
                elif time.time() - start_drowsy_time >= CLOSED_EYE_DURATION_THRESHOLD:
                    if not is_drowsy:
                        primary_alert.play()
                        alert_count += 1
                        is_drowsy = True
                        drowsiness_detected = True
                        display_text.append("DROWSINESS DETECTED!")
                    if alert_count > 3:
                        secondary_alert.play()
                        alert_count = 0
            else:
                start_drowsy_time = None
                is_drowsy = False

            # Alcohol Detection
            mar = calculate_mar(landmarks)
            head_yaw, head_pitch = get_head_pose(landmarks)
            if alcohol_detection_logic(mar, head_yaw, head_pitch):
                alcohol_alert.play()
                alcohol_detected = True
                display_text.append("ALCOHOL DETECTED!")

            # Display detection info
            display_text.append(f"EAR: {avg_ear:.2f}")
            display_text.append(f"MAR: {mar:.2f}")
            display_text.append(f"Head Yaw: {head_yaw:.2f} | Head Pitch: {head_pitch:.2f}")

    # Overlay text on the video feed
    for i, text in enumerate(display_text):
        cv2.putText(frame, text, (10, 50 + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    # Display video feed
    cv2.imshow('Driver Monitoring System', frame)

    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
pygame.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
